In [ ]:

!pip install --quiet --upgrade wrds pandas psycopg2-binary
print("Installed!  Runtime → Restart runtime, then re‑run your imports.")


In [ ]:

import wrds, pandas as pd, numpy as np

db = wrds.Connection()


In [ ]:

sp500_idx = db.raw_sql("""
    SELECT date AS caldt,          -- keep the “caldt” alias if you like
           spindx
    FROM   crsp.dsi
    ORDER  BY date
""")


sp500_idx['caldt'] = pd.to_datetime(sp500_idx['caldt'])


sp500_idx = (sp500_idx
             .rename(columns={'spindx': 'sp500_close'})
             .set_index('caldt')
             .sort_index())

print(sp500_idx.head())

# fra 1 Jan 1985 onward
sp500_idx = sp500_idx.loc['1985-01-01':].copy()


print(sp500_idx.head(3))

In [ ]:
sp500_idx = (db.get_table('crsp', 'dsi', columns=['date', 'spindx'])
               .rename(columns={'spindx': 'sp500_close'})
               .set_index('date')
               .sort_index())
print(sp500_idx)

In [ ]:
# 0)
sp_members = db.raw_sql("""
    SELECT permno,
           start  AS enter_date,
           COALESCE(ending, '9999-12-31') AS exit_date
    FROM   crsp.dsp500list
""")

# 1)
links = db.raw_sql("""
    SELECT lpermno AS permno,
           gvkey
    FROM   crsp.ccmxpf_linktable
    WHERE  linktype IN ('LU','LC')
      AND  linkprim IN ('P','C')
      AND  gvkey IS NOT NULL
""").drop_duplicates('permno')

# 2)
gics = db.raw_sql("""
    SELECT gvkey,
           gsector
    FROM   comp.company
    WHERE  gsector IS NOT NULL
""")

# 3)
memb_sec = (sp_members
            .merge(links, on='permno', how='left')
            .merge(gics,  on='gvkey',  how='left')
            .dropna(subset=['gsector']))

# 4)
permno_list = ', '.join(str(int(x)) for x in memb_sec['permno'].unique())

crsp_daily = db.raw_sql(f"""
    SELECT permno,
           date,
           prc,
           shrout,
           ret
    FROM   crsp.dsf
    WHERE  permno IN ({permno_list})
""")

crsp_daily['mktcap'] = crsp_daily['prc'].abs() * crsp_daily['shrout']

# 5)
crsp_daily = (crsp_daily
              .merge(memb_sec, on='permno', how='left')
              .query('enter_date <= date <= exit_date')
              .dropna(subset=['gsector']))

# 6)
sect_daily_ret = (
    crsp_daily
      .assign(weight=lambda d:
              d['mktcap'] / d.groupby(['date','gsector'])['mktcap'].transform('sum'))
      .assign(wret=lambda d: d['weight'] * d['ret'])
      .groupby(['date','gsector'])['wret'].sum()
      .reset_index()
)

# 7)
pivot_ret = (sect_daily_ret
             .pivot(index='date', columns='gsector', values='wret')
             .sort_index())

sect_idx = ((1 + pivot_ret.fillna(0)).cumprod() * 100) \
             .rename(columns={
                 '10':'Energy','15':'Materials','20':'Industrials',
                 '25':'ConsDiscr','30':'ConsStaples','35':'HealthCare',
                 '40':'Financials','45':'InfoTech','50':'CommServices',
                 '55':'Utilities','60':'RealEstate'
             }) \
             .astype(float)

# 8)
sect_idx.to_parquet('sp500_sector_indices.parquet')
print(sect_idx.head())

In [ ]:

sect_idx.to_csv('sp500_sector_indices.csv')
sect_idx.to_excel('sp500_sector_indices.xlsx')


from google.colab import files
files.download('/content/sp500_sector_indices.csv')
files.download('/content/sp500_sector_indices.xlsx')


In [ ]:

sp500_idx.to_csv('sp500_close_daily.csv')
sp500_idx.to_excel('sp500_close_daily.xlsx')


from google.colab import files
files.download('/content/sp500_close_daily.csv')
files.download('/content/sp500_close_daily.xlsx')

